# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_to_load='../output_data/cities.csv'


city_data=pd.read_csv(file_to_load)
city_data.head()

,City_ID,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lng,Max Temperature (F),Wind Speed (mph)
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
max_hum=city_data['Humidity (%)'].max()
max_hum

290

In [4]:
# Configure gmaps with API key:
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations: 
city_data=city_data.dropna() # drop NaN values
locations=city_data[['Lat', 'Lng']].astype(float)

# Convert humidity level to float and store:
humidity_rate=city_data["Humidity (%)"].astype(float)


In [6]:
fig=gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=290,
                                 point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
city_data_narrow=city_data[(city_data["Max Temperature (F)"] < 85) &
                             (city_data["Max Temperature (F)"] > 60) &
                             (city_data["Wind Speed (mph)"] < 15) &
                             (city_data["Cloudiness (%)"] < 30) &
                             (city_data["Humidity (%)"] < 30)]
city_data_narrow=city_data_narrow.dropna() 
city_data_narrow.count()

City_ID                5
City                   5
Cloudiness (%)         5
Country                5
Date                   5
Humidity (%)           5
Lat                    5
Lng                    5
Max Temperature (F)    5
Wind Speed (mph)       5
dtype: int64

In [8]:
city_data_narrow.head()

,City_ID,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lng,Max Temperature (F),Wind Speed (mph)
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99
367,367,pampierstad,0,ZA,1558378859,23,-27.78,24.69,67.02,10.38
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
# Set up lists to hold response info:
hotel_name=[]         
city=[]         
country=[]      
lat=[]          
lng=[]          

# Loop through the list of cities and perform a request for data on each and add to corresponding lists:

count=1 # set counter to start value 1 for printouts of the current set count

print('Beginning Data Retrieval')     
print('-----------------------------')
for i in range(len(city_data_narrow)) : 
   
    lati=str(city_data_narrow.iloc[i, 3])    
    lngi=str(city_data_narrow.iloc[i, 4])
    
    target_coordinates=lati + ', ' + lngi  
    target_search='Hotel'
    target_radius=5000
    target_type='lodging'

    # Set up a parameters dictionary:
    params={
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': g_key
    }

    # Base url:
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # Run a request using the above params dictionary:
    response=requests.get(base_url, params=params)
    #print(response.url)
    
    # Convert response to json:
    places_data=response.json()
    #print(json.dumps(places_data, indent=4, sort_keys=True))
    
    # Print user feedback: 
    print('Looking for Hotels near: ' + str(city_data_narrow.iloc[i, 0]) + ' | Record ' + str(count) + ' of Total ' + str(len(city_data_narrow)))
     
    # Add requested data to  result lists:
    try:                                             # ignore errors in response
        name.append(places_data['results'][0]['name'])
        city.append(city_data_narrow.iloc[i, 0])
        country.append(city_data_narrow.iloc[i, 1])
        lat.append(places_data['results'][0]['geometry']['location']['lat'])
        lng.append(places_data['results'][0]['geometry']['location']['lng'])
    except IndexError:
        print('No Hotel Near Location... Skipping...') # in case one of the responses comes back with error "IndexError"
                                                           # which could be no hotel near location 
  
    count=count+1
    
# Print Ending Log Statement:
print('-------------------------------')
print('Data Retrieval Complete')
print('-------------------------------')

Beginning Data Retrieval
-----------------------------
Looking for Hotels near: 230 | Record 1 of Total 5
No Hotel Near Location... Skipping...
Looking for Hotels near: 259 | Record 2 of Total 5
No Hotel Near Location... Skipping...
Looking for Hotels near: 324 | Record 3 of Total 5
No Hotel Near Location... Skipping...
Looking for Hotels near: 367 | Record 4 of Total 5
No Hotel Near Location... Skipping...
Looking for Hotels near: 444 | Record 5 of Total 5
No Hotel Near Location... Skipping...
-------------------------------
Data Retrieval Complete
-------------------------------


In [21]:
hotel_dict={
    'City': city,
    'Country': country,
    'Lat': lat,
    'Lng': lng,
    'Hotel Name': hotel_name
}
hotel_df=pd.DataFrame(hotel_dict)
hotel_df.count()

City          0
Country       0
Lat           0
Lng           0
Hotel Name    0
dtype: int64

In [22]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
figure_layout={
    'width': '955px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig=gmaps.figure(layout=figure_layout)

# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

fig=gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))